# Testing triggers issues

In [5]:
from utils import locate_edges
from pathlib import Path

trigger_channel = 'MISC005'
MEG_DATA_PATH = Path('/home/co/data/meg_distraction/240606')

import mne
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import pandas as pd
%matplotlib qt
matplotlib.use('Qt5Agg')

file_path_meg = MEG_DATA_PATH / "run7_8_raw.fif"

raw = mne.io.read_raw_fif(file_path_meg, preload=True, allow_maxshield=True)

# Crop it using the audio triggers 
stop_triggers = mne.find_events(raw)

# Crop the raw file after the 3rd stop trigger
raw.crop(tmin=0, tmax=stop_triggers[3][0]/raw.info['sfreq'])


Opening raw data file /home/co/data/meg_distraction/240606/run7_8_raw.fif...
    Read a total of 13 projection items:
        generated with dossp-2.1 (1 x 306)  idle
        generated with dossp-2.1 (1 x 306)  idle
        generated with dossp-2.1 (1 x 306)  idle
        generated with dossp-2.1 (1 x 306)  idle
        generated with dossp-2.1 (1 x 306)  idle
        generated with dossp-2.1 (1 x 306)  idle
        generated with dossp-2.1 (1 x 306)  idle
        generated with dossp-2.1 (1 x 306)  idle
        generated with dossp-2.1 (1 x 306)  idle
        generated with dossp-2.1 (1 x 306)  idle
        generated with dossp-2.1 (1 x 306)  idle
        generated with dossp-2.1 (1 x 306)  idle
        generated with dossp-2.1 (1 x 306)  idle
    Range : 11000 ... 609999 =     11.000 ...   609.999 secs
Ready.
Reading 0 ... 598999  =      0.000 ...   598.999 secs...


/tmp/ipykernel_23607/3780706.py:17: RuntimeWarning: This file contains raw Internal Active Shielding data. It may be distorted. Elekta recommends it be run through MaxFilter to produce reliable results. Consider closing the file and running MaxFilter on the data.
  raw = mne.io.read_raw_fif(file_path_meg, preload=True, allow_maxshield=True)


6 events found on stim channel STI101
Event IDs: [63]


<Raw | run7_8_raw.fif, 340 x 332411 (332.4 s), ~868.8 MB, data loaded>

In [6]:
picks = mne.pick_channels(raw.info['ch_names'], include=[trigger_channel])
#print(raw.info['ch_names'])
misc_data, times = raw[picks,:]

triggers = misc_data.T



In [7]:

#%%
# Define threshold and convert to int
# Do this but that works: for i in range(0.1, 1, 0.1):

for i in range(1, 10):
    triggers_bin = 1 * (triggers > i*0.1) 

    x, _ = locate_edges(triggers_bin)
    x = x / 1000 # in seconds
    y = np.ones(len(x)) 
    print(y.shape)


(773,)
(771,)
(768,)
(766,)
(766,)
(766,)
(765,)
(764,)
(763,)


In [10]:

#%%
#print(locate_edges(triggers_bin))
triggers_bin = 1 * (triggers > 0.8) 

x, _ = locate_edges(triggers_bin)
x = x / 1000 # in seconds
y = np.ones(len(x)) 

print(y.shape)
#print(np.diff(x))
#print(len(x))


# plt.figure(figsize=(10,4))
# plt.plot(times,misc_data.T)
# plt.plot(x,y , 'o')
# plt.show()


#%%
# Open the metadata file to get the triggers onsets
metadata_file = MEG_DATA_PATH / "run7_new.tsv"
df = pd.read_csv(metadata_file, sep='\t')
print(df.head())

first_samp = raw.first_samp
sfreq = raw.info['sfreq']
triggers_onsets = df['onset'].values

print(triggers_onsets)
# %%

events = mne.find_events(raw)
first_trigger_time = events[1][0] / sfreq
# %%
shift = x[0] - triggers_onsets[0]
triggers_onsets = triggers_onsets + shift
plt.figure(figsize=(10,4))
plt.plot(x,y , 'o')
plt.plot(triggers_onsets, y, 'o')
plt.show()




(764,)
   Unnamed: 0  onset  duration                             trial_type
0           0   3.06      0.02         {'kind': 'word', 'word': 'le'}
1           1   3.27      0.38  {'kind': 'word', 'word': 'cinquième'}
2           2   3.71      0.29       {'kind': 'word', 'word': 'jour'}
3           3   4.40      0.36   {'kind': 'word', 'word': 'toujours'}
4           4   4.82      0.21      {'kind': 'word', 'word': 'grâce'}
[  3.06       3.27       3.71       4.4        4.82       5.09
   5.115      5.21       5.96       6.14       6.57       6.7
   6.86       7.06       7.09       7.18       7.48       7.97
   8.06       8.29       9.65       9.89      10.13      10.68
  10.92      11.81      12.09      12.98      13.2       13.4
  13.7       13.76      13.88      14.38      14.95      15.49
  15.6       17.17      17.28      17.9       18.03      18.15
  18.41      18.54      19.44      19.61      19.8       20.01
  20.12      21.35      21.52      21.91      22.19      22.37
  22.53 

qt.qpa.wayland: Wayland does not support QWindow::requestActivate()
qt.qpa.wayland: Wayland does not support QWindow::requestActivate()
